#### Time series classification

Implementation of Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline (2016, arXiv) in PyTorch by using a skorch wrapper
In this script, we are using a dataset that was originally used in the paper

Authors: Marina Georgati, Hao Miao


In [1]:
import numpy as np
import os
from sail.models.torch.conv_net import ConvNet
from sail.models.torch import dataset
import torch
from io import BytesIO
from zipfile import ZipFile
from tqdm import tqdm
from urllib.request import urlopen

In [2]:
datasets = ['Adiac']
"""
Download the time series datasets used in the paper
from http://www.timeseriesclassification.com
NonInvThorax1 and NonInvThorax2 are missing
"""
datasets_pbar = tqdm(datasets)
print(datasets_pbar)
for dataset in datasets_pbar:
    datasets_pbar.set_description("Extracting {0} data".format(dataset))
    zipurl = f"http://www.timeseriesclassification.com/aeon-toolkit/Adiac.zip"
    with urlopen(zipurl) as zipfile:
        with ZipFile(BytesIO(zipfile.read())) as zfile:
            zfile.extractall(f"data/")

    assert os.path.exists(f"data/{dataset}_TRAIN.arff"), dataset

Extracting Adiac data:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


Extracting Adiac data: 100%|██████████| 1/1 [00:00<00:00,  1.91it/s]


In [3]:
# A dictionary is created by train/test DataLoaders for the downloaded dataset
dataset_dictionary = dataset.data_dictionary("data", datasets)

Processing Adiac: 100%|██████████| 1/1 [00:00<00:00,  9.12it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


In [4]:
for dataset, dataloader in dataset_dictionary.items():
    # For using the downloaded dataset, a dataset and a dataloader are created. 
    # Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples
    print('The shape of the training dataset is:', dataloader['train'].dataset.x.shape)
    print('The shape of the test dataset is:', dataloader['test'].dataset.x.shape)
    
    time_steps = dataloader['test'].dataset.x.shape[-1] # The Number of input units
    n_classes  = len(np.unique(dataloader['test'].dataset.y)) # Number of classes
    print('Number of input units:', time_steps)
    print('Number of classes:', n_classes)
    
    # The Neural Net is initialized with fixed hyperparameters
    nn = ConvNet(time_steps, n_classes)

    # The training set is defined here, modification of the datatype might be required
    y_train = dataloader['train'].dataset.y
    y_train = np.array(y_train)
    y_train = torch.from_numpy(y_train).type(torch.LongTensor)
    
    # The testing set is defined here, modification of the datatype might be required
    y_test = dataloader['test'].dataset.y
    y_test = np.array(y_test)
    y_test = torch.from_numpy(y_test).type(torch.LongTensor)

    # Fit the model on the training dataset 
    nn.fit(dataloader['train'].dataset.x, y_train)
    # Return probability estimates for samples
    nn.predict_proba(dataloader['test'].dataset.x)
    # Return class labels for samples in X
    nn.predict(dataloader['test'].dataset.x)
    # Print score
    print('Score:', nn.score(dataloader['test'].dataset.x, y_test))

The shape of the training dataset is: (390, 176)
The shape of the test dataset is: (391, 176)
Number of input units: 176
Number of classes: 37

































  epoch    train_loss     dur
-------  ------------  ------
      1        3.5515  0.7129

































      2        3.2614  0.7283

































      3        3.1515  0.6837

































      4        3.0357  0.7082

































      5        2.8706  0.7037

































      6        2.7199  0.6931

































      7        2.5973  0.7120

































      8        2.4902  0.7006

































      9        2.4036  0.7135

































     10        2.3128  0.6890



































































































Score: 0.22506393861892582


In [5]:
# OR DIFFERENTLY , calling the wrapper from src.tsc
for dataset, dataloader in dataset_dictionary.items():
    print(dataloader['train'].dataset.x.shape)
    print(dataloader['test'].dataset.x.shape)
    
    time_steps = dataloader['test'].dataset.x.shape[-1]
    n_classes  = len(np.unique(dataloader['test'].dataset.y))
    print(time_steps, n_classes)

    # The Neural Net is initialized with fixed hyperparameters
    nn = ConvNet(time_steps, n_classes)

    y_train = dataloader['train'].dataset.y
    y_train = np.array(y_train)
    y_train = torch.from_numpy(y_train).type(torch.LongTensor)
    
    y_test = dataloader['test'].dataset.y
    y_test = np.array(y_test)
    y_test = torch.from_numpy(y_test).type(torch.LongTensor)

    nn.fit(dataloader['train'].dataset.x, y_train)
    print(nn.score(dataloader['test'].dataset.x, y_test))

(390, 176)
(391, 176)
176 37

































  epoch    train_loss     dur
-------  ------------  ------
      1        3.5603  0.7273

































      2        3.2648  0.7182

































      3        3.1521  0.6899

































      4        3.0392  0.7205

































      5        2.8902  0.6912

































      6        2.7307  0.7402

































      7        2.6005  0.6922

































      8        2.4725  0.7128

































      9        2.3983  0.7198

































     10        2.3070  0.6988

































0.29411764705882354
